In [1]:
!nvidia-smi

Sun Aug 10 06:27:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from itertools import combinations
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import ast
import random
import re

In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/menu_rec/orders_grouped_cleaned(1).csv")

In [5]:
train_df['parsed_items'] = train_df['item_name'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

def dropout_cart(cart, p=0.25):
    if len(cart) <= 2: return cart
    return [item for item in cart if random.random() > p]

order_df = train_df.groupby(['ORDER_ID', 'CUSTOMER_ID'])['parsed_items'].sum().reset_index()
order_df['parsed_items'] = order_df['parsed_items'].apply(lambda x: list(set(dropout_cart(x))))
order_df = order_df[order_df['parsed_items'].str.len() > 0]

item_sequences = order_df['parsed_items'].tolist()

In [6]:
#Word2Vec Model
w2v_model = Word2Vec(
    sentences=item_sequences,
    vector_size=50,
    window=3,
    min_count=2,
    workers=4,
    sg=1,
    seed=42
)

In [7]:
# Fine-tune with test feedback
test_df_feedback = pd.read_excel("/content/drive/MyDrive/menu_rec/test_data_question.csv (1).xlsx")

In [8]:
test_df_feedback['cart_items'] = test_df_feedback[['item1', 'item2']].values.tolist()
test_df_feedback['actual_missing_item'] = test_df_feedback['item3']

fine_tune_data = [row['cart_items'] + [row['actual_missing_item']] for _, row in test_df_feedback.iterrows()]
if fine_tune_data:
    w2v_model.build_vocab(fine_tune_data, update=True)
    w2v_model.train(fine_tune_data, total_examples=len(fine_tune_data), epochs=3)

In [9]:
#Feature Engineering + Co-occurrence
def get_cart_vector(cart):
    vectors = [w2v_model.wv[i] for i in cart if i in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

def cosine_similarity(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

item_counts = Counter([item for cart in item_sequences for item in cart])
item_popularity = pd.Series(item_counts)

cooccur = defaultdict(Counter)
for cart in item_sequences:
    for i, j in combinations(set(cart), 2):
        cooccur[i][j] += 1
        cooccur[j][i] += 1

def get_item_category(item):
    item = item.lower()
    if "fries" in item or "strips" in item or "sticks" in item:
        return "side"
    elif "drink" in item or "soda" in item or "beverage" in item or "20 oz" in item:
        return "drink"
    elif "combo" in item or "meal" in item or "feast" in item or "Pack" in item:
        return "combo"
    elif "wings" in item or "sub" in item or "crispy" in item:
        return "main"
    else:
        return "other"

def get_cart_category_vector(cart_items):
    return Counter([get_item_category(i) for i in cart_items])

In [13]:
rerank_data = []
candidate_items_list = []

for _, row in test_df_feedback.iterrows():
    cart_items = row['cart_items']
    actual = row['actual_missing_item']
    cart_vec = get_cart_vector(cart_items)
    cart_len = len(cart_items)
    cart_cat = get_cart_category_vector(cart_items)

    positive = [actual]
    frequent_items = item_popularity[item_popularity > 5].index.tolist()
    negative_candidates = list(set(frequent_items) - set(cart_items) - set(positive))
    hard_negatives = random.sample(negative_candidates, min(40, len(negative_candidates)))

    for label, item in [(1, actual)] + [(0, neg) for neg in hard_negatives]:
        if item not in w2v_model.wv:
            continue

        sims = [cosine_similarity(w2v_model.wv[item], w2v_model.wv[ci])
                for ci in cart_items if ci in w2v_model.wv]
        if not sims:
            continue

        rerank_data.append({
            'ORDER_ID': row['ORDER_ID'], # Keep for merging in Layer 1.5
            'sim_avg': np.mean(sims),
            'sim_max': np.max(sims),
            'sim_min': np.min(sims),
            'sim_std': np.std(sims),
            'sim_len': len(sims),
            'popularity': np.log1p(item_popularity.get(item, 0)),
            'cooccur_sum': sum([cooccur[item][ci] for ci in cart_items]),
            'mean_cooccur': (sum([cooccur[item][ci] for ci in cart_items]) / cart_len) if cart_len else 0,
            'cart_len': cart_len,
            'cat_mismatch': int(get_item_category(item) not in cart_cat),
            'hit': label
        })
        candidate_items_list.append(item)
rerank_df = pd.DataFrame(rerank_data)
rerank_df['candidate_item'] = candidate_items_list
exclude_cols = ['hit', 'ORDER_ID', 'candidate_item']
features = [col for col in rerank_df.columns if col not in exclude_cols]

X = rerank_df[features]
y = rerank_df['hit']

group_sizes = rerank_df.groupby('ORDER_ID').size().to_list()


In [14]:
#Train LambdaRank Model
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
group_train = [41] * (len(X_train) // 41)
model = lgb.LGBMRanker(objective="lambdarank", n_estimators=100, importance_type='gain')
model.fit(X_train[:len(group_train)*41], y_train[:len(group_train)*41], group=group_train)
print("LambdaRank model trained")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1650
[LightGBM] [Info] Number of data points in the train set: 32759, number of used features: 8
LambdaRank model trained


In [15]:
#Re-rank and evaluate
def rerank_candidates(cart_items, top_k=3):
    cart_vec = get_cart_vector(cart_items)
    cart_cat = get_cart_category_vector(cart_items)
    cart_len = len(cart_items)

    candidate_set = set()
    for ci in cart_items:
        candidate_set.update(cooccur[ci].keys())
    candidate_set -= set(cart_items)
    candidates = list(candidate_set)

    candidate_scores = []
    for item in candidates:
        if item not in w2v_model.wv:
            continue

        item_vec = w2v_model.wv[item]
        sims = [cosine_similarity(item_vec, w2v_model.wv[ci]) for ci in cart_items if ci in w2v_model.wv]
        if not sims:
            continue

        sim_avg = np.mean(sims)
        sim_max = np.max(sims)
        sim_min = np.min(sims)
        sim_std = np.std(sims)
        sim_len = len(sims)
        pop = np.log1p(item_popularity.get(item, 0))
        cooccur_sum = sum([cooccur[item][ci] for ci in cart_items])
        mean_cooccur = cooccur_sum / cart_len if cart_len else 0
        item_cat = get_item_category(item)
        cat_mismatch = int(item_cat not in cart_cat)

        features_arr = np.array([[sim_avg, sim_max, sim_min, sim_std, sim_len,
                                  pop, cooccur_sum, mean_cooccur, cart_len, cat_mismatch]])
        prob = model.predict(features_arr)[0]
        candidate_scores.append((item, prob))

    top_preds = sorted(candidate_scores, key=lambda x: x[1], reverse=True)[:top_k]
    pred_items = [item for item, _ in top_preds]
    if len(pred_items) < top_k:
        backup = [i for i in item_popularity.index if i not in cart_items]
        pred_items += backup[:top_k - len(pred_items)]
    return pred_items[:top_k]

test_df_feedback['candidate_items'] = test_df_feedback['cart_items'].apply(lambda cart: rerank_candidates(cart, top_k=10))

# Optional: check what you have generated
print(test_df_feedback[['ORDER_ID', 'cart_items', 'candidate_items']].head())

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/pyth

     ORDER_ID                                       cart_items  \
0  9351345556         [Chicken Sub Combo, Ranch Dip - Regular]   
1  3595377080       [Regular Buffalo Fries, 10 pc Spicy Wings]   
2  4071757785         [Large Buffalo Fries, 10 pc Spicy Wings]   
3  3931766769  [6 pc Grilled Wings Combo, 20 pc Grilled Wings]   
4  3739700809     [Regular Buffalo Fries, 20 pc Grilled Wings]   

                                     candidate_items  
0  [2 pc Crispy Strips, 10 pc Spicy Wings, 10 pc ...  
1  [Ranch Dip - Regular, Add 5 Spicy Wings, Fried...  
2  [Ranch Dip - Regular, Fried Corn - Regular, 6 ...  
3  [Ranch Dip - Regular, Fried Corn - Large, 8 pc...  
4  [Ranch Dip - Regular, Ranch Dip - Large, Honey...  


In [16]:
print(test_df_feedback[['ORDER_ID', 'cart_items', 'candidate_items']].head())

     ORDER_ID                                       cart_items  \
0  9351345556         [Chicken Sub Combo, Ranch Dip - Regular]   
1  3595377080       [Regular Buffalo Fries, 10 pc Spicy Wings]   
2  4071757785         [Large Buffalo Fries, 10 pc Spicy Wings]   
3  3931766769  [6 pc Grilled Wings Combo, 20 pc Grilled Wings]   
4  3739700809     [Regular Buffalo Fries, 20 pc Grilled Wings]   

                                     candidate_items  
0  [2 pc Crispy Strips, 10 pc Spicy Wings, 10 pc ...  
1  [Ranch Dip - Regular, Add 5 Spicy Wings, Fried...  
2  [Ranch Dip - Regular, Fried Corn - Regular, 6 ...  
3  [Ranch Dip - Regular, Fried Corn - Large, 8 pc...  
4  [Ranch Dip - Regular, Ranch Dip - Large, Honey...  


In [18]:
rerank_df.loc[X_val.index, 'pred_score'] = model.predict(X_val)

# Explode your test_df_with_candidates on candidate_items for merging
# Convert only if needed
if isinstance(test_df_feedback['candidate_items'].iloc[0], str):
    test_df_feedback['candidate_items'] = test_df_feedback['candidate_items'].apply(eval)

# Explode candidates into separate rows
exploded_candidates = test_df_feedback.explode('candidate_items') \
                                      .rename(columns={'candidate_items': 'candidate_item'})

# Merge predictions back
merged_df = exploded_candidates.merge(
    rerank_df[['ORDER_ID', 'candidate_item', 'pred_score']],
    on=['ORDER_ID', 'candidate_item'],
    how='left'
)


In [22]:
merged_df.head()

CUSTOMER_ID  STORE_NUMBER  DAY_NUMBER DAY_TYPE    ORDER_ID  \
0    997177535          4915           1  Weekend  9351345556   
1    997177535          4915           1  Weekend  9351345556   
2    997177535          4915           1  Weekend  9351345556   
3    997177535          4915           1  Weekend  9351345556   
4    997177535          4915           1  Weekend  9351345556   

  ORDER_CHANNEL_NAME  order_sc ORDER_SUBCHANNEL_NAME ORDER_OCCASION_NAME  \
0            Digital         1                 Kiosk                ToGo   
1            Digital         1                 Kiosk                ToGo   
2            Digital         1                 Kiosk                ToGo   
3            Digital         1                 Kiosk                ToGo   
4            Digital         1                 Kiosk                ToGo   

  CUSTOMER_TYPE              item1                item2  \
0         Guest  Chicken Sub Combo  Ranch Dip - Regular   
1         Guest  Chicken Sub Combo  Ranch Dip - Regular   
2         Guest  Chicken Sub Combo  Ranch Dip - Regular   
3         Guest  Chicken Sub Combo  Ranch Dip - Regular   
4         Guest  Chicken Sub Combo  Ranch Dip - Regular   

                     item3                                cart_items  \
0  10 pc Spicy Wings Combo  [Chicken Sub Combo, Ranch Dip - Regular]   
1  10 pc Spicy Wings Combo  [Chicken Sub Combo, Ranch Dip - Regular]   
2  10 pc Spicy Wings Combo  [Chicken Sub Combo, Ranch Dip - Regular]   
3  10 pc Spicy Wings Combo  [Chicken Sub Combo, Ranch Dip - Regular]   
4  10 pc Spicy Wings Combo  [Chicken Sub Combo, Ranch Dip - Regular]   

       actual_missing_item             candidate_item  pred_score  
0  10 pc Spicy Wings Combo         2 pc Crispy Strips    1.032175  
1  10 pc Spicy Wings Combo          10 pc Spicy Wings         NaN  
2  10 pc Spicy Wings Combo  10 pc Grilled Wings Combo         NaN  
3  10 pc Spicy Wings Combo   8 pc Grilled Wings Combo         NaN  
4  10 pc Spicy Wings Combo                Chicken Sub         NaN

In [19]:
#Evaluate
correct = 0
for _, row in test_df_feedback.iterrows():
    actual = row['actual_missing_item']
    preds = row['candidate_items'][:3]  # Use top 3 candidates from the new column
    if actual in preds:
        correct += 1

In [20]:
print(f" Final Recall@3 (LambdaRank): {correct / len(test_df_feedback) * 100:.2f}%")

 Final Recall@3 (LambdaRank): 55.30%


In [23]:
# Downloading all the csv for the next layer
# Re-rank part
rerank_df.to_csv("rerank_df_layer1.csv")
# Item-popularity
import pickle
with open("item_popularity.pkl", "wb") as f:
    pickle.dump(item_popularity, f)
# co-occur
with open("cooccur.pkl", "wb") as f:
    pickle.dump(cooccur, f)

In [20]:
test_df_feedback.to_csv("test_df_with_candidates.csv", index=False)

In [24]:
merged_df.to_csv("test_df_fb_candidates.csv")